In [1]:
import scanpy as sc
import pandas as pd

In [2]:
Wang_h_adult_and_T2D = sc.read_h5ad("/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/human_diabetic_dataset_Wang2023/Wang_h_adult_and_T2D.h5ad")

In [3]:
mouse_atlas_healthy_adult = sc.read_h5ad("/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/mouse_pancreatic_atlas/mouse_atlas_healthy_adult.h5ad")

In [6]:
Elgmal_h_adult_and_diabetic = sc.read_h5ad("/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/extra_human_datasets/Elgamal/Elgmal_h_adult_and_diabetic.h5ad")

In [4]:
orth = pd.read_table("/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/mouse_human_homology.tsv")

In [5]:
Wang_h_adult_and_T2D.obs

,disease,cell_type,organism,dataset
A0011_AAACAGCCACAGCCTG-1,Pre-T2D,alpha,Homo sapiens,Wang
A0011_AAACAGCCACTAGGTC-1,Pre-T2D,beta,Homo sapiens,Wang
A0011_AAACCAACAAATTGCT-1,Pre-T2D,beta,Homo sapiens,Wang
A0011_AAACCGAAGCAGGTGG-1,Pre-T2D,beta,Homo sapiens,Wang
A0011_AAACCGAAGTGAAGTG-1,Pre-T2D,alpha,Homo sapiens,Wang
...,...,...,...,...
C0027_TTTGTTGGTGAGGTAG-1,healthy,alpha,Homo sapiens,Wang
C0027_TTTGTTGGTGCTCCGT-1,healthy,beta,Homo sapiens,Wang
C0027_TTTGTTGGTGTGTCCC-1,healthy,beta,Homo sapiens,Wang
C0027_TTTGTTGGTTTCGCGC-1,healthy,beta,Homo sapiens,Wang


In [8]:
Wang_var = Wang_h_adult_and_T2D.var
Wang_var = Wang_var.copy()
Wang_var["name"] = Wang_var.index

In [9]:
duplicates = orth[orth.duplicated(subset="Human gene name", keep=False)]

In [10]:
unique_ids_per_group = duplicates.groupby("Human gene name")["Gene stable ID"].nunique()
groups_with_multiple_ids = unique_ids_per_group[unique_ids_per_group > 1]
genes_with_multiple_ids = unique_ids_per_group[unique_ids_per_group > 1].index.tolist()

In [11]:
filtered_orth = orth[~orth["Human gene name"].isin(genes_with_multiple_ids)]

In [12]:
orth_deduplicated = filtered_orth.drop_duplicates(subset="Human gene name", keep="first")

In [13]:
merged_df = Wang_var.merge(right=orth_deduplicated, left_on="name", right_on="Human gene name", how="left")
merged_df.index = merged_df["name"]
merged_df.index = merged_df.index.rename("EID")

In [14]:
Wang_h_adult_and_T2D.var = merged_df
Wang_h_adult_and_T2D_orth = Wang_h_adult_and_T2D[:,~Wang_h_adult_and_T2D.var['Gene stable ID'].isna()]

Wang_h_adult_and_T2D_orth.var.index = Wang_h_adult_and_T2D_orth.var["Gene stable ID"]

Wang_h_adult_and_T2D_orth.var.index = Wang_h_adult_and_T2D_orth.var.index.rename("EID")
Wang_h_adult_and_T2D_orth.var = Wang_h_adult_and_T2D_orth.var.copy()

Wang_h_adult_and_T2D_orth = Wang_h_adult_and_T2D_orth[:,Wang_h_adult_and_T2D_orth.var["Human homology type"] == "ortholog_one2one"]

/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [15]:
Elgmal_h_adult_and_diabetic_obs = Elgmal_h_adult_and_diabetic.obs

In [16]:
Elgmal_h_adult_and_diabetic_obs["Diabetes Status"] = Elgmal_h_adult_and_diabetic_obs["Diabetes Status"].replace("ND", "healthy")
Elgmal_h_adult_and_diabetic_obs["disease"] = Elgmal_h_adult_and_diabetic_obs["Diabetes Status"]

In [17]:
Elgmal_h_adult_and_diabetic_obs.rename({"Cell Type":"cell_type"}, axis = 1, inplace = True)

In [18]:
Elgmal_h_adult_and_diabetic_obs = Elgmal_h_adult_and_diabetic.obs
Elgmal_h_adult_and_diabetic_obs["dataset"] = "Elgmal"
Elgmal_h_adult_and_diabetic_obs["organism"] = "Homo sapiens"
Elgmal_h_adult_and_diabetic_obs = Elgmal_h_adult_and_diabetic_obs[["disease", "dataset", "organism", "cell_type"]]

In [19]:
Elgamal_var = Elgmal_h_adult_and_diabetic.var
Elgamal_var = Elgamal_var.copy()
Elgamal_var["name"] = Elgamal_var.index
Elgamal_var.index = Elgamal_var.index.rename("EID")

In [20]:
# Check for duplicate keys in both dataframes
print(Elgamal_var["name"].duplicated().sum())  # In Wang_var
print(orth["Human gene name"].duplicated().sum())  # In orth
merged_df.index = merged_df["name"]

0
86226


In [21]:
orth_deduplicated.index = orth_deduplicated["Human gene name"]
orth_deduplicated.index = orth_deduplicated.index.rename("EID")

In [22]:
merged_df = Elgamal_var.merge(right=orth_deduplicated, left_on="name", right_on="Human gene name", how="left")
merged_df.index = merged_df["Gene stable ID"].rename("EID")

In [23]:
Elgmal_h_adult_and_diabetic.var = merged_df
#Elgmal_h_adult_and_diabetic.var["Transcript stable ID"]

In [24]:
Elgmal_h_adult_and_diabetic = Elgmal_h_adult_and_diabetic[:,~Elgmal_h_adult_and_diabetic.var['Gene stable ID'].isna()]

Elgmal_h_adult_and_diabetic.var = Elgmal_h_adult_and_diabetic.var.copy()
Elgmal_h_adult_and_diabetic = Elgmal_h_adult_and_diabetic[:,Elgmal_h_adult_and_diabetic.var["Human homology type"] == "ortholog_one2one"]

/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [25]:
Elgmal_h_adult_and_diabetic.obs = Elgmal_h_adult_and_diabetic_obs

In [26]:
mouse_atlas_healthy_adult_obs = mouse_atlas_healthy_adult.obs

In [27]:
mouse_atlas_healthy_adult_obs["disease"] = "healthy"

In [28]:
mouse_atlas_healthy_adult.obs = mouse_atlas_healthy_adult_obs[["disease", "dataset", "organism", "cell_type"]]

In [29]:
mouse_atlas_healthy_adult.X = mouse_atlas_healthy_adult.raw.X

Preprocessing of datasets

In [30]:
sc.pp.normalize_per_cell(mouse_atlas_healthy_adult)

/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:524: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata.obs[key_n_counts] = counts_per_cell


In [31]:
sc.pp.log1p(mouse_atlas_healthy_adult)

In [32]:
sc.pp.normalize_per_cell(Wang_h_adult_and_T2D_orth)

/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:524: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs[key_n_counts] = counts_per_cell


In [33]:
sc.pp.log1p(Wang_h_adult_and_T2D_orth)

In [34]:
var = pd.DataFrame(index = Wang_h_adult_and_T2D_orth.var.index)

In [35]:
Wang_h_adult_and_T2D_orth.var = var

In [36]:
Wang_h_adult_and_T2D_orth = Wang_h_adult_and_T2D_orth[Wang_h_adult_and_T2D_orth.obs.disease != "Pre-T2D"]

In [38]:
sc.pp.normalize_per_cell(Elgmal_h_adult_and_diabetic)

/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:524: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adata.obs[key_n_counts] = counts_per_cell


In [39]:
sc.pp.log1p(Elgmal_h_adult_and_diabetic)

In [22]:
Wang_h_adult_and_T2D_orth.write_h5ad("/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/data_to_predict/wang_to_predict.h5ad")

In [40]:
adata_concat = sc.concat([mouse_atlas_healthy_adult, Elgmal_h_adult_and_diabetic, Wang_h_adult_and_T2D_orth], join="inner")

/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/anndata/_core/merge.py:942: UserWarning: Only some AnnData objects have `.raw` attribute, not concatenating `.raw` attributes.
  warn(


In [41]:
adata_extra_preT2diseaseadata_concat[adata_concat.obs.disease == "Pre-T2D"]

In [42]:
adata_extra_preT2D

View of AnnData object with n_obs × n_vars = 0 × 15457
    obs: 'disease', 'dataset', 'organism', 'cell_type', 'n_counts'

In [101]:
adata_extra_preT2D.write_h5ad("./train_data/extra_human_preT2D_new.h5ad")

In [88]:
ind = [not elm for elm in adata_concat.obs.disease == "Pre-T2D"]

In [50]:
adata_train_baseline = adata_concat

In [51]:
adata_train_baseline.obs.disease.replace({"healthy":"normal"}, inplace = True)

In [52]:
adata_train_baseline.obs.disease.unique()

array(['normal', 'T2D'], dtype=object)

In [54]:
adata_train_baseline

AnnData object with n_obs × n_vars = 204754 × 15457
    obs: 'disease', 'dataset', 'organism', 'cell_type', 'n_counts'

In [55]:
adata_train_baseline.write_h5ad("./train_data/full_datasets/train_adata_baseline.h5ad")

In [49]:
adata_concat.write_h5ad("./train_data/train_adata_baseline_deduplicated.h5ad")

In [35]:
sc.pp.highly_variable_genes(adata_concat, n_top_genes=3000)

In [36]:
adata_concat_highly_variable = adata_concat[:,adata_concat.var.highly_variable]

In [38]:
adata_concat_highly_variable.write_h5ad("./train_data/train_adata_baseline_top3000hvg_deduplicated.h5ad")

In [39]:
Wang_h_adult_and_T2D_orth_concat = Wang_h_adult_and_T2D_orth[:,Wang_h_adult_and_T2D_orth.var.index.isin(adata_concat.var.index)]

In [40]:
Wang_h_adult_and_T2D_orth_concat.var = pd.DataFrame(Wang_h_adult_and_T2D_orth_concat.var.index)

In [41]:
Wang_h_adult_and_T2D_orth_concat.var.index = Wang_h_adult_and_T2D_orth_concat.var["EID"]

In [42]:
Wang_h_adult_and_T2D_orth_concat.var = Wang_h_adult_and_T2D_orth_concat.var.drop("EID", axis=1)

In [43]:
delattr(Wang_h_adult_and_T2D_orth_concat, "uns")

In [44]:
Wang_h_adult_and_T2D_orth_concat.obs = Wang_h_adult_and_T2D_orth_concat.obs.drop("n_counts", axis=1)

In [45]:
Wang_h_adult_and_T2D_orth_concat.write_h5ad("./data_to_predict/Wang_to_predict.h5ad")

### deduplication test

In [46]:
import pandas as pd

In [47]:
genes_original = pd.read_csv("./top3000_hvg_baseline_train_data.csv")

In [51]:
genes_deduplicated = pd.DataFrame(index = adata_concat_highly_variable.var.index)

In [52]:
sum(genes_original.EID.isin(adata_concat_highly_variable.var.index))

3000

In [53]:
sum(genes_original.EID.isin(genes_deduplicated.index))/len(adata_concat_highly_variable.var)*100

100.0

### Ground truth dataset

In [51]:
mouse_atlas = sc.read_h5ad("/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/mouse_pancreatic_atlas/mouse_atlas_hrovatin.h5ad")

#### dbdb

In [52]:
dbdb = mouse_atlas[mouse_atlas.obs.dataset == "db/db"]

In [53]:
dbdb.obs.disease

index
CTTCCTTGTACCCAGC-1-MUC13640-VSG    type 2 diabetes mellitus
GTGCTTCCATTGCCTC-1-MUC13631-VSG    type 2 diabetes mellitus
ATAGGCTCATGCAGCC-1-MUC13639-VSG    type 2 diabetes mellitus
TACCTGCAGGAAAGGT-1-MUC13640-VSG    type 2 diabetes mellitus
TCGGATAGTAACCCGC-1-MUC13641-VSG    type 2 diabetes mellitus
                                             ...           
CCCTTAGTCGGAATGG-1-MUC13640-VSG    type 2 diabetes mellitus
CCCTTAGAGACAACAT-1-MUC13642-VSG    type 2 diabetes mellitus
AAATGGATCACCTCGT-1-MUC13631-VSG    type 2 diabetes mellitus
GTAGAAAAGACGGTCA-1-MUC13632-VSG    type 2 diabetes mellitus
ATGAGGGCATTCCTCG-1-MUC13632-VSG    type 2 diabetes mellitus
Name: disease, Length: 69745, dtype: category
Categories (2, object): ['type 2 diabetes mellitus', 'normal']

In [54]:
replace_dict = {
    "normal": "healthy",
    "type 2 diabetes mellitus": "T2D"
}

dbdb.obs.disease = dbdb.obs.disease.replace(replace_dict)

/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/pandas/core/generic.py:5931: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  self[name] = value


In [55]:
dbdb.obs = dbdb.obs[["disease", "dataset", "organism", "cell_type"]]

In [56]:
dbdb.var = pd.DataFrame(index=dbdb.var.index)

In [57]:
delattr(dbdb, "obsm")

In [58]:
delattr(dbdb, "uns")

In [61]:
dbdb.write_h5ad("./ground_truth/dbdb_ground_truth.h5ad")

In [63]:
pd.DataFrame(index = adata_concat_highly_variable.var.index).to_csv("../top3000_hvg_baseline_train_data_deduplicated.h5ad")

In [64]:
dbd_top3000_baseline_hvg = dbdb[:,dbdb.var.index.isin(adata_concat_highly_variable.var.index)]

In [65]:
dbd_top3000_baseline_hvg.write_h5ad("./ground_truth/dbdb_ground_truth_top3000_baseline_hvg_deduplicated.h5ad")

#### mSTZ

In [62]:
mSTZ = mouse_atlas[mouse_atlas.obs.dataset == "mSTZ"]

In [63]:
replace_dict = {
    "normal": "healthy",
    "type 2 diabetes mellitus": "T2D"
}

mSTZ.obs.disease = mSTZ.obs.disease.replace(replace_dict)

/d/hpc/home/mo6643/miniconda3/envs/msc1/lib/python3.10/site-packages/pandas/core/generic.py:5931: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  self[name] = value


In [64]:
mSTZ.obs = mSTZ.obs[["disease", "dataset", "organism", "cell_type"]]

In [65]:
mSTZ.var = pd.DataFrame(index=mSTZ.var.index)

In [66]:
delattr(mSTZ, "obsm")

In [67]:
delattr(mSTZ, "uns")

In [68]:
mSTZ.write_h5ad("./ground_truth/mSTZ_ground_truth.h5ad")

In [72]:
mSTZ

AnnData object with n_obs × n_vars = 49545 × 31334
    obs: 'disease', 'dataset', 'organism', 'cell_type'

In [73]:
mSTZ_top3000_baseline_hvg = mSTZ[:,mSTZ.var.index.isin(adata_concat_highly_variable.var.index)]

In [74]:
mSTZ_top3000_baseline_hvg.write_h5ad("./ground_truth/mSTZ_ground_truth_top3000_baseline_hvg_deduplicated.h5ad")

### Wnag to predict

In [75]:
Wang_h_adult_and_T2D_orth_concat[:,Wang_h_adult_and_T2D_orth_concat.var.index.isin(adata_concat_highly_variable.var.index)].write_h5ad("./data_to_predict/Wang_to_predict_top3000_baseline_hvg_deduplicated.h5ad")